In [1]:
## normaly, the centralize DP -> perturbing the outcomes of the Algorithms
## LDP -> assume that the analyst is malicious but have a trusted data curator or not =)) so you may made your data DP before send it
## LDP has a significant drawnback:
## accuracy of query results in the local model is typically orders of magnitude lower for the same amount of the privacy cost (epsilon)

In [2]:
## The RR is based on fliping two independent coins => getting the uncertainty of the true answer
import numpy as np

def rand_resp_sales(response):
    truthful_response = response == 'Sales'
    # first coin flip
    if np.random.randint(0, 2) == 0:
        # answer truthfully
        return truthful_response
    else:
        # answer randomly (second coin flip)
        return np.random.randint(0, 2) == 0


In [4]:
## ask 200 people who do work in sales to respond using randomized response and look at the results
import pandas as pd

pd.Series([rand_resp_sales('Sales') for i in range(200)]).value_counts()

True     147
False     53
dtype: int64

In [5]:
import matplotlib.pyplot as plt
adult = pd.read_csv("adult_with_pii.csv")
adult.head()

,Name,DOB,SSN,Zip,Workclass,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Hours per week,Country,Target,Age,Capital Gain,Capital Loss
0,Karrie Trusslove,9/7/1967,732-14-6110,64152,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,56,2174,0
1,Brandise Tripony,6/7/1988,150-19-2766,61523,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35,0,0
2,Brenn McNeely,8/6/1991,725-59-9860,95668,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,32,0,0
3,Dorry Poter,4/6/2009,659-57-4974,25503,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,14,0,0
4,Dick Honnan,9/16/1951,220-93-3811,75387,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,72,0,0


In [6]:
# In the real system => each respondant run the rand_resp locally and submit their RR to the data curator 

responses = [rand_resp_sales(r) for r in adult['Occupation']]
pd.Series(responses).value_counts()

False    22649
True      9914
dtype: int64

In [19]:
## How to estimate the actual number of salespeople in the dataset
len(adult[adult["Occupation"] == "Sales"])

3650

In [20]:
## => the RR is still true but it does not reveal a lot information about the data, just for statistical result we can see the
## data

## To get the true number of salespeople, we need to analyze the randomnes in the randomized response algorithm
## and estimate how many time the "yes" response from the true salespeople

## prob 1/2 the respondant responds randomly => then prob 1/2 random response is a "yes"
## => the prob that a respondant reponds yes bby random change rather than bcs they are a salesperson is 1/2*1/2 => we may expect the
## 1/4 of total response to be "fake yes"

responses = [rand_resp_sales(r) for r in adult['Occupation']]
# we expect 1/4 of the responses to be "yes" based entirely on the coin flip
# these are "fake" yesses
fake_yesses = len(responses)/4
# the total number of yesses recorded
num_yesses = np.sum([1 if r else 0 for r in responses])
# the number of "real" yesses is the total number of yesses minus the fake yesses
true_yesses = num_yesses - fake_yesses

In [26]:
true_yesses

1767.25

In [27]:
## Since we split the respondants into "truth" and "random" groups randomly (by the first flip of coin) => we can hope that
## there are roughly the same number of salespeople in both groups.
# true_yesses estimates the total number of yesses in the "truth" group
# we estimate the total number of yesses for both groups by doubling
rr_result = true_yesses*2
rr_result

3534.5

In [28]:
true_result = np.sum(adult['Occupation'] == 'Sales')
true_result


3650

In [ ]:
## We can see that the rr_result & the true result almost the same